##### Copyright 2020 Die TensorFlow-Autoren.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Das sequentielle Modell

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/sequential_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/sequential_model.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Notizbuch herunterladen</a></td>
</table>

## Konfiguration

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Wann wird ein sequentielles Modell verwendet?

Ein `Sequential` Modell ist für **einen einfachen Stapel von Schichten** geeignet, wobei jede Schicht **genau einen Eingangstensor und einen Ausgangstensor hat** .

Schematisch das folgende `Sequential` Modell:

In [0]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

entspricht dieser Funktion:

In [0]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

Ein sequentielles Modell ist **nicht geeignet,** wenn:

- Ihr Modell verfügt über mehrere Eingänge oder mehrere Ausgänge
- Jede Ihrer Ebenen verfügt über mehrere Eingänge oder mehrere Ausgänge
- Sie müssen Layer-Sharing durchführen
- Sie möchten eine nichtlineare Topologie (z. B. eine Restverbindung, ein Mehrzweigmodell).

## Erstellen eines sequentiellen Modells

Sie können ein sequentielles Modell erstellen, indem Sie eine Liste von Ebenen an den sequentiellen Konstruktor übergeben:

In [0]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

Auf seine Ebenen kann über das `layers` zugegriffen werden:

In [0]:
model.layers

Sie können ein sequentielles Modell auch schrittweise über die Methode `add()` erstellen:

In [0]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

Beachten Sie, dass es auch eine entsprechende `pop()` -Methode zum Entfernen von Ebenen gibt: Ein sequentielles Modell verhält sich sehr ähnlich wie eine Liste von Ebenen.

In [0]:
model.pop()
print(len(model.layers))  # 2

Beachten Sie auch , dass die sequenzielle Konstruktor ein akzeptiert `name` Argument, wie jede Schicht oder Modell in Keras. Dies ist nützlich, um TensorBoard-Diagramme mit semantisch aussagekräftigen Namen zu versehen.

In [0]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## Festlegen der Eingabeform im Voraus

Im Allgemeinen müssen alle Ebenen in Keras die Form ihrer Eingaben kennen, um ihre Gewichte erstellen zu können. Wenn Sie also eine Ebene wie diese erstellen, hat sie zunächst keine Gewichte:

In [0]:
layer = layers.Dense(3)
layer.weights  # Empty

Es erstellt seine Gewichte beim ersten Aufruf einer Eingabe, da die Form der Gewichte von der Form der Eingaben abhängt:

In [0]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

Dies gilt natürlich auch für sequentielle Modelle. Wenn Sie ein sequentielles Modell ohne Eingabeform instanziieren, wird es nicht "erstellt": Es hat keine Gewichte (und das Aufrufen von `model.weights` führt zu einem Fehler, der genau dies angibt). Die Gewichte werden erstellt, wenn das Modell zum ersten Mal Eingabedaten sieht:

In [0]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Sobald ein Modell "erstellt" ist, können Sie seine `summary()` -Methode aufrufen, um seinen Inhalt anzuzeigen:

In [0]:
model.summary()

Es kann jedoch sehr nützlich sein, wenn Sie ein sequentielles Modell inkrementell erstellen, um die Zusammenfassung des bisherigen Modells einschließlich der aktuellen Ausgabeform anzeigen zu können. In diesem Fall sollten Sie Ihr Modell , indem ein Start `Input` Objekt zu Ihrem Modell, so dass es seine Eingangsform von Anfang an weiß:

In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Beachten Sie, dass das `Input` nicht als Teil angezeigt wird `model.layers` , da es nicht eine Schicht ist:

In [0]:
model.layers

Eine einfache Alternative besteht darin, einfach ein `input_shape` Argument an Ihre erste Ebene zu übergeben:

In [0]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Modelle, die mit einer solchen vordefinierten Eingabeform erstellt wurden, haben immer Gewichte (noch bevor Daten angezeigt werden) und haben immer eine definierte Ausgabeform.

Im Allgemeinen wird empfohlen, die Eingabeform eines sequentiellen Modells immer im Voraus anzugeben, wenn Sie wissen, um was es sich handelt.

## Ein gängiger Debugging-Workflow: `add()` + `summary()`

When building a new Sequential architecture, it's useful to incrementally stack layers with `add()` and frequently print model summaries. For instance, this enables you to monitor how a stack of `Conv2D` and `MaxPooling2D` layers is downsampling image feature maps:

In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Sehr praktisch, oder?


## Was tun, wenn Sie ein Modell haben?

Sobald Ihre Modellarchitektur fertig ist, möchten Sie:

- Trainieren Sie Ihr Modell, bewerten Sie es und führen Sie Inferenzen durch. Lesen Sie unseren [Leitfaden zur Schulung und Bewertung mit den integrierten Schleifen](https://www.tensorflow.org/guide/keras/train_and_evaluate/)
- Speichern Sie Ihr Modell auf der Festplatte und stellen Sie es wieder her. Weitere Informationen finden Sie in unserer [Anleitung zum Serialisieren und Speichern](https://www.tensorflow.org/guide/keras/save_and_serialize/) .
- Beschleunigen Sie das Modelltraining, indem Sie mehrere GPUs einsetzen. Lesen Sie unseren [Leitfaden für Multi-GPU- und verteilte Schulungen](distributed_training) .

## Merkmalsextraktion mit einem sequentiellen Modell

Sobald ein sequentielles Modell erstellt wurde, verhält es sich wie ein [funktionales API-Modell](https://www.tensorflow.org/guide/keras/functional/) . Dies bedeutet, dass jede Ebene ein `input` und `output` hat. Diese Attribute können verwendet werden, um nette Dinge zu erledigen, z. B. schnell ein Modell zu erstellen, das die Ausgaben aller Zwischenebenen in einem sequentiellen Modell extrahiert:

In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

Hier ist ein ähnliches Beispiel, das nur Features aus einer Ebene extrahiert:

In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Übertragen Sie das Lernen mit einem sequentiellen Modell

Transferlernen besteht darin, die unteren Schichten in einem Modell einzufrieren und nur die oberen Schichten zu trainieren. Wenn Sie nicht damit vertraut sind, lesen Sie unbedingt unseren [Leitfaden zum Lerntransfer](https://www.tensorflow.org/guide/keras/transfer_learning/) .

Hier sind zwei gängige Blaupausen für das Transferlernen mit sequentiellen Modellen.

First, let's say that you have a Sequential model, and you want to freeze all layers except the last one. In this case, you would simply iterate over `model.layers` and set `layer.trainable = False` on each layer, except the last one. Like this:

```python
model = keras.Sequential([     keras.Input(shape=(784))     layers.Dense(32, activation='relu'),     layers.Dense(32, activation='relu'),     layers.Dense(32, activation='relu'),     layers.Dense(10), ])  # Presumably you would want to first load pre-trained weights. model.load_weights(...)  # Freeze all layers except the last one. for layer in model.layers[:-1]:   layer.trainable = False  # Recompile and train (this will only update the weights of the last layer). model.compile(...) model.fit(...)
```

Ein weiterer gängiger Entwurf besteht darin, ein sequentielles Modell zu verwenden, um ein vorab trainiertes Modell und einige frisch initialisierte Klassifizierungsebenen zu stapeln. So was:

```python
# Load a convolutional base with pre-trained weights base_model = keras.applications.Xception(     weights='imagenet',     include_top=False,     pooling='avg')  # Freeze the base model base_model.trainable = False  # Use a Sequential model to add a trainable classifier on top model = keras.Sequential([     base_model,     layers.Dense(1000), ])  # Compile & train model.compile(...) model.fit(...)
```

Wenn Sie das Lernen übertragen, werden Sie wahrscheinlich häufig diese beiden Muster verwenden.

Das ist ungefähr alles, was Sie über sequentielle Modelle wissen müssen!

Weitere Informationen zum Erstellen von Modellen in Keras finden Sie unter:

- [Anleitung zur funktionalen API](https://www.tensorflow.org/guide/keras/functional/)
- [Anleitung zum Erstellen neuer Ebenen und Modelle über Unterklassen](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)